# 25: MAXIMA VEROSIMILITUD

In [1]:
import numpy as np
from IPython.display import display, Math

### Definir la función de entorno L(B)

In [2]:
display(Math(r'L(\beta)=\sum_{i=1}^n P_i^{y_i}(1-Pi)^{1-y_i}'))

<IPython.core.display.Math object>

In [3]:
def likelihood(y, pi): # y es un vector de 1 y 0 (casos observados) y pi es un vector con las probabilidades de cada observación
    
    total_sum = 1
    sum_in = list(range(1, len(y) + 1)) # crea una lista con el siguiente formato [1,2,3,4,5,...,N]
    
    for i in range(len(y)):
        sum_in[i] = np.where(y[i] == 1, pi[i], 1 - pi[i]) # actualiza la lista con la probabilidad de exito si ha habido exito o la 
        total_sum = total_sum * sum_in[i]                 # de fracaso (1 - Pi) en caso de fracaso 
        
    return total_sum

In [4]:
# Ejemplo de como usar esta función

vector_observaciones = [1,1,0,1,0,0,1,0] # un 1 significa exito y un 0 significa fracaso
vector_probabilidades = [0.5,0.6,0.3,0.2,0.8,0.5,0.6,0.5] # cada elemento es la posibilidad de exito para esa observación

likelihood(vector_observaciones, vector_probabilidades)

0.0012599999999999998

### Definir la probabilidad de cada una de las observaciones

In [5]:
display(Math(r'P_i = P(x_i) = \frac{1}{1+e^{-\sum_{j=0}^k\beta_j\cdot x_{ij}}} '))

<IPython.core.display.Math object>

In [6]:
def logitprobs(X, beta): # X es una matriz donde cada columna representa los distintos parametros de una observación y cada fila
                         # representa una única observación con todos sus valores para cada elemento. Beta es un vector
                         # con el valor Bi para cada uno de los parametros de la observación
    
    n_rows = np.shape(X)[0] # obtiene el número de observaciones
    n_cols = np.shape(X)[1] # obtiene el número de parametros de cada observación
    
    pi = list(range(1, n_rows + 1)) # crea una lista con los valores [1,2,3,4,...,N (NÚMERO DE OBSERVACIONES)] 
    expon = list(range(1, n_rows + 1)) # crea una lista con los valores [1,2,3,4,...,N (NÚMERO DE PARAMETROS)] 
    
    for i in range(n_rows):
        expon[i] = 0
        for j in range(n_cols):
            ex = X[i][j] * beta[j] # multiplicamos un parametro de una observación por su respectiva beta dentro del loop
            expon[i] = ex + expon[i]
        with np.errstate(divide = "ignore", invalid = "ignore"):
            pi[i] = 1 / (1 + np.exp(-expon[i]))
            
    return pi # retorna un vector de probabilidades para cada observación

In [ ]:
# Ejemplo de como usar esta función

betas = [2.8,0.9,0.004,1.2,1,0.6,2.8,1.4]
observaciones = np.array([[1,1,0,0,1,1,0,1], # matriz de 8 parametros con 3 observaciones
                          [1,0,1,1,1,0,0,1],
                          [0,0,1,1,0,0,1,0]])

logitprobs(observaciones, betas)

### Definir la matriz diagonal W

In [8]:
display(Math(r'W= diag(P_i \cdot (1-P_i))_{i=1}^n'))

<IPython.core.display.Math object>

In [9]:
def findW(pi): # pi es un vector de probabiliadades de exito para cada observación
    
    n = len(pi)
    W = np.zeros(n * n).reshape(n, n) # construye un vector de N^2 elementos 0 y lo convertimos en matriz de N x N de dimensiones
    
    for i in range(n):
        W[i,i] = pi[i] * (1 - pi[i]) # en cada uno de los elementos de la diagonal se pone esa fórmula
        W[i,i].astype(float)
        
    return W # retorna una matriz de ceros con diagonal de izquierda a derecha

In [15]:
# Ejemplo de como usar esta función

findW(vector_probabilidades)

array([[0.25, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.24, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.21, 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.16, 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.16, 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.25, 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.24, 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.25]])

### Definir función para obtener betas

Esta función va autilizar el resto de funciones que hemos creado con anterioridad para obtener el valor de las betas de los parametros del modelo.

In [17]:
display(Math(r"\beta_{n+1} = \beta_n -\frac{f(\beta_n)}{f'(\beta_n)}"))
display(Math(r"f(\beta) = X(Y-P)"))
display(Math(r"f'(\beta) = XWX^T"))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [25]:
def logistics(X, Y, limit): # X es una matriz donde cada columna representa un parametro de la observación, y cada fila representa una
                            # única observación con todos sus parametros. La Y es un vector con cada una de las observaciones, siendo
                            # 1 éxito y 0 el fracaso. Por último, limit es un valor númerico que establecemos como threshold para diseñar
                            # la mejor estimación posible 
    
    from numpy import linalg
    
    nrow = np.shape(X)[0] # sacamos el número de observaciones que tenemos. También valdria hacer un len(Y)
    bias = np.ones(nrow).reshape(nrow, 1) # hacemos un vector de tantos 1 como observaciones. Luego lo hacemos matriz de dimension N x 1
    X_new = np.append(X, bias, axis = 1) # hacemos una nueva matriz que será igual a la X añadiendo la matríz 'BIAS' que hemos creado 
    ncol = np.shape(X_new)[1] # sacamos el numero de parametros que tenemos para cada observación + 1 (el bias)
    beta = np.zeros(ncol).reshape(ncol, 1) # hacemos un vector de tantos 0 como parametros + 1. Luego lo hacemos matriz de dimension N x 1
    root_dif = np.array(range(1, ncol + 1)).reshape(ncol, 1) # crea un vector de 1 hasta el número de parametros + 1 y luego lo convierte
    iter_i = 10000                                           # en matriz de dimensiones N x 1
    
    while(iter_i > limit):
        print("Iter: i " + str(iter_i) + ", limit: " + str(limit))
        pi = logitprobs(X_new, beta) # utilizamos la función para crear un vector de probabilidades para cada observación
        print("Pi:" + str(pi))
        W = findW(pi) # utiliamos la función para crear la matrix diagonal
        print("W:" + str(W))
        num = (np.transpose(np.matrix(X_new)) * np.matrix(Y - np.transpose(pi)).transpose())
        den = (np.matrix(np.transpose(X_new)) * np.matrix(W) * np.matrix(X_new))
        root_dif = np.array(linalg.inv(den) * num)
        beta = beta + root_dif
        print("Beta: " + str(beta))
        iter_i = np.sum(root_dif * root_dif)
        ll = likelihood(Y, pi)
        
    return beta

In [26]:
# Ejemplo de como usar esta función

y_observaciones = [0,0,1,0,0,1,0,1,0,1,1,1,0,1,0,1,1] # 17 observaciones
x_parametros = np.array([[1,2,3,4,5],
                         [0,0,2,4,6],
                         [1,1,2,0,0],
                         [5,2,1,1,1],
                         [3,3,2,1,2],
                         [0,0,0,4,1],
                         [2,2,1,1,4],
                         [2,1,1,0,0],
                         [0,0,0,1,0],
                         [4,4,5,2,1],
                         [2,2,3,4,5],
                         [1,1,1,2,1],
                         [1,1,2,2,2],
                         [4,4,2,3,5],
                         [5,5,2,1,0],
                         [0,2,0,2,1],
                         [1,1,2,2,3]]) # 17 observaciones con 5 parametros por cada observación

logistics(x_parametros, y_observaciones, 0.005)

Iter: i 10000, limit: 0.005
Pi:[array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5])]
W:[[0.25 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.  ]
 [0.   0.25 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.  ]
 [0.   0.   0.25 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.  ]
 [0.   0.   0.   0.25 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.  ]
 [0.   0.   0.   0.   0.25 0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.25 0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.25 0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.25 0.   0.   0.   0.   0.   0.
  0.   0.   0.  ]
 [0.   0.   0.   0.   0.

C:\Users\Usuario\AppData\Local\Temp\ipykernel_18956\3768832947.py:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  W[i,i]=pi[i]*(1-pi[i])


array([[-0.40751702],
       [ 0.25582606],
       [ 0.37705381],
       [ 0.54289133],
       [-0.53691236],
       [-0.13511335]])

El mejor modelo para estos datos sería el siguiente:

In [38]:
display(Math(r' P = \frac{1}{1+e^{-(-0.40751702+0.25582606 * X_1 + 0.37705381 * X_2 + 0.54289133 * X_3 - 0.53691236 * X_4 - 0.13511335 * X_5)}}'))

<IPython.core.display.Math object>